In [1]:
import numpy as np
import pygame
import random
import time
import srat 
# Размеры окна
CELL_SIZE = 50

# Инициализация pygame
pygame.init()

# Загрузка изображений
TREE_IMAGE = pygame.image.load('tree.png')  # Замените на путь к вашему изображению дерева
FIRE_IMAGE = pygame.image.load('fire.png')  # Замените на путь к вашему изображению огня

# Масштабируем изображения до нужного размера
TREE_IMAGE = pygame.transform.scale(TREE_IMAGE, (CELL_SIZE, CELL_SIZE))
FIRE_IMAGE = pygame.transform.scale(FIRE_IMAGE, (CELL_SIZE, CELL_SIZE))

# Цвет для пустых клеток
EMPTY_COLOR = (255, 255, 255)  # Белый - пустая клетка

# Функция для инициализации поля
def create_forest(n):
    return np.full((n, n), None)  # Массив n x n, заполненный значением None

# Функция для обновления состояния клеток
def update_forest(forest, epsilon, theta):
    n = forest.shape[0]
    
    # Список для клеток, которые начнут гореть на следующем шаге
    fire_spread = []

    # Список для клеток, которые сгорят на следующем шаге
    to_burn = []

    # Процесс распространения огня
    for i in range(n):
        for j in range(n):
            if forest[i, j] == 'Fire':
                # Соседи
                neighbors = get_neighbors(i, j, n)
                for ni, nj in neighbors:
                    if forest[ni, nj] == 'Tree':  # Если сосед - дерево, он начнет гореть
                        fire_spread.append((ni, nj))  # Добавляем его в список для горения
                # Горящее дерево добавляем в список на следующий шаг
                to_burn.append((i, j))

    # Обновляем клетки, которые должны загореться на следующем шаге
    for ni, nj in fire_spread:
        if forest[ni, nj] == 'Tree':  # Убедимся, что клетка еще не горит
            forest[ni, nj] = 'Fire'

    # Горящие деревья становятся пустыми на следующем шаге
    for i, j in to_burn:
        forest[i, j] = None
    for i in range(n):
        for j in range(n):
            if forest[i, j] == 'Tree' and random.random() < theta:
                forest[i, j] = 'Fire'
    # Процесс появления деревьев с вероятностью epsilon (теперь в конце функции)
    for i in range(n):
        for j in range(n):
            if forest[i, j] is None:  # Только пустая клетка может стать деревом
                if random.random() < epsilon:
                    forest[i, j] = 'Tree'

# Функция для получения соседей клетки
def get_neighbors(i, j, n):
    neighbors = []
    if i > 0: neighbors.append((i-1, j))  # Верх
    if i < n-1: neighbors.append((i+1, j))  # Низ
    if j > 0: neighbors.append((i, j-1))  # Лево
    if j < n-1: neighbors.append((i, j+1))  # Право
    return neighbors

# Функция для рисования поля
def draw_forest(screen, forest, n):
    for i in range(n):
        for j in range(n):
            if forest[i, j] == 'Tree':
                screen.blit(TREE_IMAGE, (j * CELL_SIZE, i * CELL_SIZE))  # Отображаем дерево
            elif forest[i, j] == 'Fire':
                screen.blit(FIRE_IMAGE, (j * CELL_SIZE, i * CELL_SIZE))  # Отображаем огонь

# Основная функция игры
def main():
    # Параметры
    n = int(input("Введите размер поля n: "))
    epsilon = float(input("Введите вероятность появления дерева ε (от 0 до 1): "))
    theta = float(input("Введите вероятность удара молнии θ (от 0 до 1): "))
    
    # Размер окна
    width = height = n * CELL_SIZE
    screen = pygame.display.set_mode((width, height))
    pygame.display.set_caption("Лесной Пиздец")

    # Инициализация поля
    forest = create_forest(n)

    # Главный цикл
    running = True
    clock = pygame.time.Clock()
    while running:
        screen.fill(EMPTY_COLOR)
        
        # Обновляем состояние леса
        update_forest(forest, epsilon, theta)

        # Рисуем лес
        draw_forest(screen, forest, n)
        
        # Обновляем экран
        pygame.display.flip()

        # Проверка событий
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

        # Пауза, чтобы симуляция была видимой (0.5 секунды задержка)
        time.sleep(1.5)
        
        # Ограничиваем частоту кадров
        clock.tick(10)

    pygame.quit()

if __name__ == "__main__":
    main()


pygame 2.6.1 (SDL 2.28.4, Python 3.13.2)
Hello from the pygame community. https://www.pygame.org/contribute.html


Введите размер поля n:  6
Введите вероятность появления дерева ε (от 0 до 1):  0.5
Введите вероятность удара молнии θ (от 0 до 1):  0.3
